In [145]:
# imports
import pandas as pd
import json
import os
from IPython.core.debugger import set_trace
# change directory
# os.chdir(os.path.dirname(os.getcwd()))

In [146]:
os.chdir('c:\\Users\\Techie\\Documents\\soenke\\Solaranlagen\\src')

In [147]:
# settings for Lottie (Animations)

data_points_url = 'https://assets5.lottiefiles.com/packages/lf20_6dboqita.json'
wondering_url = 'https://assets4.lottiefiles.com/packages/lf20_pcoatxlk.json'
typing_url = 'https://assets1.lottiefiles.com/temporary_files/r5WAZZ.json'
reportlist_url='https://assets9.lottiefiles.com/packages/lf20_dq6whaxi.json'
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))

In [148]:
# load settings
with open('../references/settings/user_settings.json','r') as f:
    settings = json.load(f)
with open('../references/settings/extended_settings.json','r') as f:
    settings.update(json.load(f))
with open('../references/settings/dev_settings.json','r') as f:
    settings.update(json.load(f))

# compute the real compensation per kWh fed into the grid in 0,01*€
scale = settings['project_info']['scale']
compensation = (
    settings['financial']['base_compensation'][scale]
    *0.985**(pd.to_datetime(settings['project_info']['planned_commission_date']).month-9)
)

In [149]:
# set application title
application_name = 'BraSolar'

In [150]:
# import libraries & components

# import custom components
# in the future the source will need to be changed (add d07_visualisation in front of 'components')
from components import start_button, no_fig, your_fig, description

# 
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc 
from dash_extensions import Lottie
import pandas as pd
import os
from dash import no_update
from dash.exceptions import PreventUpdate
from time import sleep, perf_counter
import geopandas as gpd


In [151]:
# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SANDSTONE])
app.title = application_name

app.layout = dbc.Container([
    # row one
    dbc.Row([
        # row one col one
        dbc.Col([
            # logo
            dbc.Card([
                dbc.CardImg(src='../assets/logo.png',
                style={'height':'50px', 'width': '50px'}
                )
            ],className='mb-2'),
            # name and link to website
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink('Sönke Maibach',target='_blank',  # _blank opens a new tab on click
                    href='https://mabikono.de')
                ])
            ],style={'height':'inherit'}),
        ], width=2),
        # row one col two
        dbc.Col([
            # title and short info
            dbc.Card([
                dbc.CardBody([
                    html.H4('{}'.format(application_name)),
                    html.P('Find out if your chosen locations are suitable for solar power.')
                ])
            ], color = 'info',style={'height':'18vh'}),
        ], width=6),
        # row one col three
        # Dark/Light Mode and Language Dropdown
        dbc.Col([
            dbc.Card([
                dbc.Button('Turn on light',color='primary', id='theme_switch')
            ])
        ],width=2)
    ], className='mb-2 mr-2 ml-2 h-25'),
    # row two
    dbc.Row([
        # row two col one
        # description
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options,url=wondering_url, width='40%', height='40%')),
                dbc.CardBody([
                    html.H4('What is going on here?',className='card-title'),
                    description
                ])
            ]),
        ], width=4, style={'height':'inherit'}),
        # row two col two
        # inputs and start button
        dbc.Col([
            dbc.Card([
                # card header for the inputs
                dbc.CardHeader(
                    Lottie(options=options,url=typing_url, height='25%', width='25%')
                ),
                dbc.CardBody([
                    html.H4('Please tell me what interests you'),
                    # input field for location
                    html.I("Choose a city, region or country."),
                    html.Br(),
                    dcc.Input(id="location_input", type="search",
                    value="Duesseldorf",
                    style={'marginRight':'20px'}),
                    html.Br(), html.Br(),

                    
                    
                    # radioitems resolution
                    html.I("Choose the resolution. A higher resolution means more data points (DP) will be analyzed."),
                    html.Br(),
                    dcc.RadioItems(id='resolution_radio',
                        options=[
                            {'label':' Fastest (~100 DP)','value':200},
                            {'label':' Balanced (~400 DP)','value':800},
                            {'label':' Very detailed (~1000 DP)','value':2000},
                        ],
                        value=200,
                        labelStyle={'display': 'block'}
                    ),

                    # Household Size input
                    html.I('How many people live in your household? Children count as 0.5.'),
                    html.Br(),
                    dcc.Input(id="household_input", type="number",
                    value=3,
                    style={'marginRight':'20px'}),
                    html.Br(), html.Br(),

                    # start button
                    html.Div('',id='old_request'),
                    start_button,

                    # choose output to plot
                    html.I('Choose what you want to see.'), html.Br(),
                    dcc.RadioItems(
                        options=[
                            {'label':'Maximum electricity output','value':'maxout'},
                            {'label':'Maximum savings/financial return','value':'maxroi'},
                        ],
                        value='maxout',
                        labelStyle={'display':'block'},
                        id='to_plot_radio'
                    )

                ])
            ]),
        ], width=4, style={'height':'inherit'}),
        # row two col three
        # status updates
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(url=reportlist_url,options=options,height='20%',width='20%')),
                dbc.CardBody([
                    html.H4('Status report'),
                    html.Div('Status updates will be displayed after starting the analysis.',id='started_status'),
                    html.Div('',id='loc_ref_status'),
                    html.Div('',id='get_weather_status'),
                    html.Div('',id='loc_ref_block_status'),
                    html.Div('',id='weather_pivot_status'),
                    html.Div('',id='topo_get_status'),
                    html.Div('',id='terrain_block_status'),
                    html.Div('',id='sungeo_status'),
                    html.Div('',id='pollution_status'),
                    html.Div('',id='need_status'),
                    html.Div('',id='angle_status'),
                    html.Div('',id='output_status'),
                    html.Div('',id='maxout_status'),
                    html.Div('',id='maxroi_status'),
                    html.Div('',id='plotted_status'),
                ],id='status_card')
            ]),
        ], width=4, style={'height':'inherit'}),
    ],className='m-2 mt-4 mb-4 h-15'),
    # row three 
    dbc.Row([
        # row three col one
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='map_graph',figure=no_fig),
                ])
            ]),
        ], width=12,style={}),
        # row three col two
        # additional graph next to map

    ],className='m-2',style={"height":"100%"}),

    dbc.Row([
        # hidden data storage
        # store min and max dates, datetimes
        dcc.Store(id='minmax_date_store'), 
        dcc.Store(id='minmax_datetime_store'),

        # store the process_id
        dcc.Store(id='process_id_store',data=['sample_id']),
    ],className='mt-2 mr-2 ml-2',style={}),
    

], fluid=True, style={
    
    'background-image':'url("/assets/bg_solar.jpg")',
    'background-repeat':'no-repeat',
    'background-size':'cover',
    'background-position': 'bottom-center',
    'background-attachment':'fixed',
    'font-family': "calibri"
    })

Callbacks

In [152]:
# utility imports
import os
from time import sleep, perf_counter
from ast import literal_eval

In [153]:
# reset outputs when changing user input
# check if it's a completely new request and inform the user if that's the case
@app.callback(
    Output('start_button','n_clicks'),
    Output('old_request','children'),
    Output('process_id_store','data'),
    Input('location_input','value'),
    Input('resolution_radio','value'),
    Input('household_input','value')
)
def reset_analysis(loc,res,hh):
    print('clicks reset')

    # check if the results of the request are available in the database 
    # get the process id
    process_id = str(loc).lower()+'_'+str(res)
    # get the save paths
    max_out_path = '../data/03_processed/max_out_{}.pqt'.format(process_id)
    max_roi_path = '../data/03_processed/max_roi_{}.pqt'.format(process_id)
    polygons_path = '../references/polygons_{}.json'.format(process_id)

    if os.path.exists(max_out_path) and os.path.exists(max_roi_path) and os.path.exists(polygons_path):
        user_info = 'This was requested before. Output should be available quickly.'
    else:
        user_info = 'This is a new request. Please be patient.'

    return (
        # reset n_clicks
        0,
        # inform user if a long wait time is to be expected
        user_info,
        # set process_id
        [process_id],
    )

In [154]:
# start button func
# triggered by clicking the start button
# outputs text to the 'started_status' wrapper

@app.callback(
    Output('started_status','children'),
    Input('start_button','n_clicks'),
    State('old_request','children')

)
def start_analysis(clicks,req):
    if clicks == 0:
        return ('Status updates will be displayed after starting the analysis.')
    else:
        return '0% done - Analysis started'

In [155]:
# when the started_status updates, make sure a location reference polygon is available
# after checking (or requesting) update the corresponding status div

# imports
from d01_data.get_address_data import load_save_reference as loadsaveref

@app.callback(
    Output('loc_ref_status','children'),
    Input('started_status','children'),
    State('location_input','value')
)
def check_locref(started,loc):
    if started[0] == 'S':
        print('locref triggered but no update')
        return ''
    else:
        starttime = perf_counter()
        locref = loadsaveref(loc)
        print('locref triggered and work done')
        elapsed = perf_counter()-starttime
        return ('1% done - Location understood in '+f" {elapsed:0.2f} seconds.")

In [156]:
# after checking the location reference, check if weather data is available
# if it's missing, request it

# imports
from d01_data.get_weather_data import getload_weather

@app.callback(
    Output('get_weather_status','children'),
    Input('loc_ref_status','children'),
    State('location_input','value'),
    State('resolution_radio','value'),
    State('process_id_store','data')
)
def get_weather(locref,loc,res,process_id):
    if locref == '':
        print('get weather triggered, return empty string')
        return ''
    else:
        starttime = perf_counter()
        weather_df = getload_weather(process_id[0],location=str(loc),resolution=res)

        print('weather checked, id: {}'.format(process_id))

        elapsed = perf_counter() - starttime
        return ('30% done - Weather data loaded in '+f'{elapsed:0.2f} seconds')

In [157]:
# start the id management process
# create and save an id_management_df with info about allowed and disallowed ids + reasons for blocking


# imports
from d00_utils.id_management import id_manage_df, id_allow_df, update_id_block_df, updateblocks_idmanage_df
import shapely.geometry as sg

@app.callback(
    Output('loc_ref_block_status','children'),
    Input('get_weather_status','children'), # this should trigger when the weather status changes,
    State('location_input','value'),
    State('resolution_radio','value'),
    State('process_id_store','data')
)
def loc_ref_block(prev_status,loc,res,process_id):
    if prev_status =='':
        print('loc ref block triggered, nothing returned')
        return ''
    else:
        starttime = perf_counter()

        process_id = process_id[0]
        
        # load the weather df, but only id col
        ids_df = pd.read_parquet('../data/01_raw/weather_{}.pqt'.format(process_id))['id_tuple'].unique()
        ids_df = pd.DataFrame(ids_df)
        ids_df.columns = ['id_tuple']
        print('ids df is a '+str(type(ids_df)))

        global df
        df = ids_df

        # create dfs to manage ids and blocks
        idmanage_df = id_manage_df(ids_df,id_col='id_tuple')
        idmanage_df.id_tuple = [literal_eval(tup) for tup in idmanage_df.id_tuple]
        block_df = pd.DataFrame(columns=['id_tuple','blocked','block reason'])

        print('pre point')
        # create a geometry col

        # update the block_df: block locations outside of the requested region
        block_df = update_id_block_df(
            ids_df,block_df,
            'outside of',
            loadsaveref(str(loc).lower()),
            'outside of '+loc,
            block_col='geometry',
            insights=False)
        print('blocks updated')
        # update the main management df
        idmanage_df = updateblocks_idmanage_df(block_df, idmanage_df)
        # change id_tuple dtype to string, then save the main df
        idmanage_df.id_tuple = idmanage_df.id_tuple.astype(str)
        idmanage_df.to_parquet('../references/id_management/id_manage_df_{}.pqt'.format(process_id))

        print('blocks by locref done')
        elapsed = perf_counter()-starttime
        return ('32% done - Weather data double-checked in '+f'{elapsed:0.2f} seconds')



In [158]:
# pivot the weather df

# imports
import datetime as dt

@app.callback(
    Output('weather_pivot_status','children'),
    Output('minmax_date_store','data'),
    Output('minmax_datetime_store','data'),
    Input('loc_ref_block_status','children'), # trigger when the step before is done
    State('location_input','value'),
    State('resolution_radio','value'),
    State('process_id_store','data')
)
def weather_pivot(locref_status,loc,res,process_id):
    if locref_status == '':
        return (
            # return empty string for the status div
            '',
            # return no update for the minmax date and datetime stores
            no_update,no_update
        )
    else:
        starttime = perf_counter()

        # recreate process_id
        process_id = process_id[0]
        # load weather
        weather_df =pd.read_parquet('../data/01_raw/weather_{}.pqt'.format(process_id))
        # load allowed ids from idmanage_df
        idmanage_df = pd.read_parquet('../references/id_management/id_manage_df_{}.pqt'.format(process_id))
        idmanage_df = idmanage_df.loc[idmanage_df.blocked ==False].id_tuple

        # filter weather by allowed ids
        weather_df = weather_df.loc[weather_df.id_tuple.isin(idmanage_df)]

        # pivot the weather df so that all data corresponding to a specific time and location are in a single row
        weather_df = weather_df.pivot(['id_tuple','date'],'parameter','value').reset_index()
        # create separate cols for date and datetime
        weather_df['datetime'] = weather_df.date
        weather_df['date'] = [x.date() for x in weather_df.date]

        # simplify the col names of weather_df
        # take the col names, create empty list for new names
        cols = weather_df.columns
        new_cols = []
        # iterate over the vol names
        for name in cols:
            pos = name.find(':')
            # check if there are units given, if yes drop them
            if pos >0:
                name = name[:pos]
            # collect new col names
            new_cols.append(name)
        # rename the cols of weather_df
        weather_df.columns=new_cols

        # new col am which states if a row has a MEST in the morning
        weather_df.loc[:,'pm'] = [
            (x.hour>10) & (x.hour<23)
            for x
            in weather_df.datetime]

        # get the min and max datetime as well as min and max dates
        minmax_dates = {'min':weather_df.date.iat[0], 'max':weather_df.date.iat[-1]}
        minmax_datetimes = {'min':weather_df.datetime.iat[0], 'max':weather_df.datetime.iat[-1]}


        # save the pivoted and filtered weather df as weather_df_pivoted
        weather_df.to_parquet('../data/01_raw/weather_pivoted_{}.pqt'.format(process_id))

        # print update for devs
        print('weather pivoted')

        # get needed timespan and return update to user
        elapsed = perf_counter()-starttime
    
        return (
            # return the status update
            ('38% done - Weather data reformatted in '+f'{elapsed:0.2f} seconds'),
            # return minmax dates and minmax datetimes
            minmax_dates, minmax_datetimes
        )

In [159]:
# analyse the topographical surroundings
# make sure topo data is available, if not, request and save it
# topo data will be saved forever
# analyse the environment for uneven ground or north facing slopes
# use the analysis to block locations
'''this could be improved in the future: better understanding 
    of the specific locations in the weather df could lead to less requested and 
    lower the need for requests
    also there is room for improvement by putting the async functions into their own module
    '''

# imports
from d01_data.get_topo_data import run_requests

@app.callback(
    Output('topo_get_status','children'),
    Input('weather_pivot_status','children'), # should be triggered when the weather data is pivoted
    State('location_input','value'),
    State('resolution_radio','value'),
    State('process_id_store','data')
)
def get_topo(weather_status,loc,res,process_id):
    if weather_status =='':
        print('no weather data, waiting')
        return '' # return empty if the last step is not done
    else:
        print('timer started for requests')
        # start the timer
        starttime = perf_counter() 

        # access process_id
        process_id = process_id[0]

        # load the idmanage_df, this will be needed later on
        idmanage_df = pd.read_parquet('../references/id_management/id_manage_df_{}.pqt'.format(process_id))

        # check if topo data relevant to the user request is available
        topo_path = '../data/02_intermediate/topo_data_' + process_id +'.pqt'
        if os.path.exists(topo_path):
            # if the data is available, load it
            print('topo available')
        else:
            # request topo data
            print('topo requested')
            # get allowed ids
            ids_allowed = idmanage_df.loc[idmanage_df.blocked ==False][['id_tuple']]
            # change dtype to tuple
            ids_allowed.id_tuple = ([
                literal_eval(tup)
                for tup in ids_allowed.id_tuple
            ])
            # inform devs of dtype
            print('thats the type: {}'.format(type(ids_allowed.iat[0,0])))
            # run the request, this will also save the topo df
            res = run_requests(ids_allowed.id_tuple,process_id)
            print('requests finished')

        # stop the timer
        elapsed = perf_counter()-starttime
        return ('45% done - Topographical data loaded in '+f'{elapsed:0.2f} seconds')

In [160]:
# block locations based on topographical conditions
# if there's a north facing slope or uneven ground at the location it will be blocked

# imports
from d00_utils.id_management import update_id_block_df, updateblocks_idmanage_df

@app.callback(
    Output('terrain_block_status','children'),
    # triggers after making sure topo data is available
    Input('topo_get_status','children'),
    State('process_id_store','data')
)
def block_by_topo(status,process_id):
    if status == '':
        return ''
    else:
        # start the timer
        starttime = perf_counter()
        print('topo block started')
        # get process id
        process_id = process_id[0]

        # load topo data and idmanage_df
        topo_path = '../data/02_intermediate/topo_data_' + process_id +'.pqt'
        topo_df = pd.read_parquet(topo_path)
        idmanage_df = pd.read_parquet('../references/id_management/id_manage_df_{}.pqt'.format(process_id))

        # create a block_df as needed by the update_id_block_df function
        block_df = idmanage_df.loc[idmanage_df.blocked]

        # block locations with uneven ground
        # if the standard deviation of the elevation is greater than the allowed value, the location will be blocked
        # if you know what you are doing, you can change the max value by accessing /references/dev_settings.json
        #   and changing the values in elevation_std_ref
        
        # compare the elevation std with the max allowed std based on the project scope
        # load the reference value first
        # get the scale from the settings
        scale = settings['project_info']['scale']
        # get the reference value from the settings which fits the scale
        elevation_std_ref = settings['topo_references']['elevation_std'][scale]

        # update the block_df based on the elevation std
        block_df = update_id_block_df(
            topo_df,
            block_df,
            'greater than',
            elevation_std_ref,
            'ground needs to be more even',
            block_col='elevation_std'
        )


        # compare the slope at a given location with the allowed reference. 
        # if the gradient is less than the relevance, there's a north facing slope. the loc will be blocked
        # get the reference value from the settings which fits the scale
        min_ns_grad_ref = settings['topo_references']['min_ns_grad'][scale]
        # update the block_df 
        # update blocks if the ground is facing north
        block_df = update_id_block_df(
            topo_df,
            block_df,
            'less than',
            min_ns_grad_ref,
            'north facing slope',
            block_col='ns_gradient'
        )

        # update the idmanage_df and save it
        idmanage_df = updateblocks_idmanage_df(block_df, idmanage_df)
        idmanage_df.to_parquet('../references/id_management/id_manage_df_{}.pqt'.format(process_id))

        # stop the timer
        elapsed = perf_counter()-starttime
        return ('53% done - Topographical conditions analyzed in '+f'{elapsed:0.2f} seconds')
        

In [161]:
# compute the position of the sun relative to locations
# also compute the mean aod since the last rain

# imports 
from d02_intermediate.geometry3d import sun_geo
from d02_intermediate.est_albedo import albedo
from d02_intermediate.weather_int import rain_prep

@app.callback(
    Output('sungeo_status','children'),
    Output('pollution_status','children'),
    Input('terrain_block_status','children'),
    State('process_id_store','data'),
    State('minmax_datetime_store','data')
)
def sun_geometry(status,process_id,minmax_datetime):
    if status == '':
        return '',''
    else:
        # start timer
        starttime = perf_counter()

        # get process_id
        process_id =process_id[0]

        # load weather_df
        weather_df = pd.read_parquet('../data/01_raw/weather_pivoted_{}.pqt'.format(process_id))
        # load idmanage_df
        idmanage_df = pd.read_parquet('../references/id_management/id_manage_df_{}.pqt'.format(process_id))
        # filter weather_df by allowed ids
        weather_df = weather_df.loc[
            weather_df.id_tuple.isin(
                idmanage_df.loc[idmanage_df.blocked==False,'id_tuple']
            )
        ]
        weather_df.reset_index(inplace=True,drop=True)

        # change dtype of id_tuple col to tuple
        weather_df.id_tuple = [
            literal_eval(tup)
            for tup in weather_df.id_tuple
        ]

        # get gemotry data
        weather_df.loc[:,'sungeo'] = [
            sun_geo(x[0],x[1],d)
            for x,d
            in zip(
                weather_df.id_tuple,
                weather_df.datetime
            )]

        # unpack the relevant data
        weather_df.loc[:,'azimut'] = [x['azimut'] for x in weather_df['sungeo']]
        weather_df.loc[:,'sunheight'] = [x['sunheight refracted'] for x in weather_df['sungeo']]
        weather_df.loc[:,'albedo'] = [albedo(x[0],x[1]) for x in weather_df.id_tuple]
        # drop the unpacked col
        weather_df.drop('sungeo',inplace=True,axis=1)

        # stop the timer
        elapsed1 = perf_counter() - starttime
        # start new timer
        starttime = perf_counter()


        # get reference values from the settings and dcc.Store
        # look up estimate of time since last rain in hours if there's no data available
        rain_est = settings['losses']['rain_reference']
        # look up earliest and latest available datetime
        global dt_min, dt_max
        dt_min, dt_max = pd.to_datetime(minmax_datetime['min']), pd.to_datetime(minmax_datetime['max'])

        # the following code utilizes window functions to estimate the average amount of dust in the atmosphere
        # we begin by looking up/estimating the time since the last and until the next rain event relative to a given datetime 
        # the last time tells us which values have to be included in the average
        # the next time tells us at what point a new window has to be opened for the windows function to work properly

        # create new col for weather_df to find the datetime of the next rain
        # use the current time as next rain time if raining and then backfill, if there are still na values, use the latest possible datetime
        weather_df.loc[:,'next_rain'] = pd.Series([
            rain_prep(precip, datetime, dt_min)
            for precip, datetime
            in zip( weather_df.precip_1h,
                    weather_df.datetime)
            ])
        weather_df.next_rain = weather_df.next_rain.fillna(method='bfill').fillna(dt_max)

        # create new col for weather_df to find the datetime of the last rain
        # use the current time as last rain time if raining, then forward fill
        # if data is missing the last datetime will be imputed based on the rain estimate from the settings
        weather_df.loc[:,'last_rain'] = pd.Series([
            rain_prep(precip, datetime,dt_min,next=False)
            for precip, datetime
            in zip( weather_df.precip_1h,
                    weather_df.datetime)
            ])
        weather_df.last_rain = weather_df.last_rain.fillna(method='ffill').fillna(dt_min-dt.timedelta(hours=rain_est))

        # compute the time since the last rain in hours
        # might be possible to simplify this, previous attempts took at least 10 times as long to complete
        weather_df.loc[:,'hrs_since_rain'] = [
            int((now-last).seconds/3600 + (now-last).days*24)
            for now, last
            in zip( weather_df.datetime,
                    weather_df.last_rain)
        ]

        # finally compute the mean pollution (aod_mean) since the last rain event
        weather_df.loc[:,'aod_mean'] = (
            weather_df
            # groupby location and next rain event to create correct windows
            .groupby(['id_tuple','next_rain'])
            # create expanding windows so only values before or at current datetime will be included in average
            .expanding()
            # get the mean values
            .agg({'total_aod_550nm':'mean'})
            # reset index
            .reset_index()
            # simplify column names
            .drop('level_2',axis=1)
            # keep only the mean values of the pollution to add to weather_df
            .total_aod_550nm
        )

        # drop columns from weather_df which served their purpose
        weather_df.drop(['total_aod_550nm','next_rain','last_rain','precip_1h'],axis=1,inplace=True)

        
        # change id_tuple col dtype back to string
        weather_df.id_tuple = weather_df.id_tuple.astype(str)
        # save the weather_df with the new data
        weather_df.to_parquet('../data/02_intermediate/weather_pollution_{}.pqt'.format(process_id))

        # stop the second timer
        elapsed2 = perf_counter() - starttime

        return (
            ('59% done - Relative position of the sun calculated in '+f'{elapsed1:0.2f} seconds'),
            ('63% done - Average pollution of the atmosphere calculated in '+f'{elapsed2:0.2f} seconds'),
        )


In [162]:
# after the geometric data and pollution averages are computed, estimate the energy need

# imports
from d02_intermediate.est_consumption import consumption_by_date, consumption_ampm

@app.callback(
    Output('need_status','children'),
    Input('sungeo_status','children'), # trigger when the sun geometry is computed
    State('household_input','value'),
    State('process_id_store','data'),
    State('minmax_date_store','data')
)
def get_need(status,hh,process_id,minmax_dates):
    if status == '':
        return ''
    else:
        # start the timer
        starttime = perf_counter()
        # get the process id
        process_id = process_id[0]


        # compute the energy needs per half day
        # instantiate an empty list do collect dfs for each day
        need_list = []
        for day in (
            pd.date_range(
                minmax_dates['min'],
                minmax_dates['max']
            )
        ):
            # compute the daily total, use it to compute need for half days
            a = consumption_by_date(
                day,
                settings['energy_need']['monthly_winter'],
                settings['energy_need']['monthly_winter'],
                hh
            )
            b = consumption_ampm(
                a,
                settings['energy_need']['night'] + settings['energy_need']['morning'],
                settings['energy_need']['afternoon'] + settings['energy_need']['evening']
            )
            # save the half days in a df, collect the dfs
            b = pd.DataFrame(b,columns=['need','pm'])
            b['date']=day.date()
            need_list.append(b)
        # create a full df with all the need data
        need_df = pd.concat(need_list,axis=0,ignore_index=True)
        # set the index to make joining later on easier
        need_df.set_index(['date','pm'],inplace=True)
        # save the need_df
        need_df.to_parquet('../data/02_intermediate/need_df_{}.pqt'.format(process_id))

        # stop the timer
        elapsed = perf_counter() -starttime
        return ('66% done - Energy consumption estimated in '+f'{elapsed:0.2f} seconds')

In [163]:
# create suggestions for tilt and alignment values based on the need df
# compute the angle of the sun beams to the panels with all suggested tilts & alignments

# imports
from d00_utils.tiltalign_adjust import tilt_options,align_options
from math import atan, degrees
from d01_data.get_weather_data import get_solar_irradiance
from d02_intermediate.geometry3d import inc_angle_by_degrees as inc_angle
from d02_intermediate.est_incidence_on_panel import radiation_incidence_on_panel as incidence_on_panel

@app.callback(
    Output('angle_status','children'),
    Input('need_status','children'), # trigger when XYZ
    State('process_id_store','data')
)
def get_suggestions(status,process_id):
    if status == '':
        return ''
    else:
        # start the timer
        starttime = perf_counter()
        # get the process id
        process_id = process_id[0]

        # look up power need settings
        need_settings = settings['energy_need']
        # generate suggestions for tilt and alignment values based on the need
        # tilt suggestions are based on the ratio of need in the summer vs winter
        tilt_suggs = tilt_options(
            need_settings['monthly_summer'],
            need_settings['monthly_winter']
        )
        # alignment suggestions are based on the consumption profile per day
        align_suggs = align_options(
            need_settings['night'],
            need_settings['morning'],
            need_settings['afternoon'],
            need_settings['evening'],
        )

        # load weather_df
        weather_df = pd.read_parquet('../data/02_intermediate/weather_pollution_{}.pqt'.format(process_id))

        # add alignment and tilt options to each location and datetime

        # add a tilt column which contains a string with the tilt options
        weather_df.loc[:,'tilt'] = '-'.join([str(x) for x in tilt_suggs])
        # split the string into a list
        weather_df.tilt = weather_df.tilt.str.split('-')
        # create a row for each item in the list
        weather_df = weather_df.explode('tilt')
        # repeat the process for the alignment suggestions
        weather_df.loc[:,'alignment'] = '-'.join([str(x) for x in align_suggs])
        weather_df.alignment = weather_df.alignment.str.split('-')
        weather_df = weather_df.explode('alignment').reset_index(drop=True)
        # change the dtype of the tilts and alignments to integer
        # this will lose a bit of details but it's fine
        weather_df.tilt = weather_df.tilt.astype('int')
        weather_df.alignment = weather_df.alignment.astype('int')

        # compute the angle of incidence for each location and tilt alignment combination
        # compute angle of incidence
        weather_df.loc[:,'angle'] = [inc_angle(
                a,
                s,
                tilt,
                alignment
            ) 
            for a,s,tilt,alignment
            in zip(
                weather_df.azimut,
                weather_df.sunheight,
                weather_df.tilt,
                weather_df.alignment)]
        # compute the solar irradiance for each date
        weather_df.loc[:,'solar_irradiance'] = [get_solar_irradiance(x) for x in weather_df.date]
        # compute the incidence on the panel
        weather_df.loc[:,'incidence'] = [
            incidence_on_panel(glo,dir,dif,irr,sh,tilt,albedo,angle)
            for glo,dir,dif,irr,sh,tilt,albedo,angle
            in zip(
                weather_df.global_rad,
                weather_df.direct_rad,
                weather_df.diffuse_rad,
                weather_df.solar_irradiance,
                weather_df.sunheight,
                weather_df.tilt,
                weather_df.albedo,
                weather_df.angle
            )]

        # drop used data
        weather_df.drop(['direct_rad','diffuse_rad','solar_irradiance','sunheight','albedo','angle','azimut'],axis=1,inplace=True)

        # save the weather_df
        weather_df.to_parquet('../data/02_intermediate/weather_df_angles_{}.pqt'.format(process_id))

        # stop the timer
        elapsed = perf_counter() -starttime
        return ('81% done - Angle of sun beams on solar panel calculated in '+f'{elapsed:0.2f} seconds')

In [164]:
# compute the losses and output per m² panel area for each location, tilt and alignment

# imports
from d02_intermediate.weather_int import est_snowdepth_panel, est_paneltemp, loss_by_snow, loss_by_temp, est_soiling_loss_data


@app.callback(
    Output('output_status','children'),
    Input('angle_status','children'), # trigger when XYZ
    State('process_id_store','data')
)
def get_need(status,process_id):
    if status == '':
        return ''
    else:
        # start the timer
        starttime = perf_counter()
        # get the process id
        process_id = process_id[0]

        # load weather_df
        weather_df = pd.read_parquet('../data/02_intermediate/weather_df_angles_{}.pqt'.format(process_id))

        # compute loss by snow on panel
        weather_df.loc[:,'loss_snow'] = \
            [loss_by_snow(\
                est_snowdepth_panel(x)) 
            for x
            in weather_df.snow_depth]

        # look up technical losses and panel_efficiency
        panel_info = settings['panel_info']

        # compute loss by temperature of the panel
        weather_df.loc[:,'loss_temp'] = \
            [loss_by_temp(\
                est_paneltemp(temp, wind),panel_info['temperature_coefficient']) 
            for temp, wind
            in zip(
                weather_df.t_2m,
                weather_df.wind_speed_10m) ]

        # compute loss by soiling on panel
        from d02_intermediate.weather_int import est_soil_loss_value as slvalue
        weather_df.loc[:,'loss_soiling'] = \
            [ slvalue(tilt,hrs_since,aod_mean)
            for tilt, hrs_since, aod_mean
            in zip(weather_df.tilt,weather_df.hrs_since_rain,weather_df.aod_mean)]

        # drop used data 
        weather_df.drop(['snow_depth', 'wind_speed_10m','t_2m','aod_mean','hrs_since_rain'],axis=1,inplace=True)

        # compute a loss column with the total losses by all high variance factors combined
        weather_df.loc[:,'loss_highvar'] = [
            1-(1-l1)*(1-l2)*(1-l3)
            for l1,l2,l3 
            in zip(
                weather_df.loss_snow,
                weather_df.loss_soiling,
                weather_df.loss_temp
            )
        ]


        # look up technical loss
        technical_loss = settings['losses']['loss_technical']
        # compute an output col with output after highvar losses in w/m²
        weather_df.loc[:,'output'] = [
            incidence*(1-losses)* panel_info['efficiency'] * (1-technical_loss)
            for incidence, losses
            in zip(
                weather_df.incidence,
                weather_df.loss_highvar
            )
        ]

        # save the weather_df
        weather_df.to_parquet('../data/03_processed/weather_df_output_{}'.format(process_id))

        # stop the timer
        elapsed = perf_counter() -starttime
        return ('90% done - Possible outputs calculated in '+f'{elapsed:0.2f} seconds')

In [165]:
# create custom aggregation function
# will be used to find the smallest total panel area that still leads to the requested ratio of self sufficient days
def agg_percentile_finder(series):
    '''
    returns the percentile, only works if the series is sorted (ascending)'''
    # find the position of the value corresponding to the percentile
    pos = ceil(len(series)*settings['project_info']['sufficiency_ratio'])-1
    return series.iloc[pos]

# func to compute kwp from total panel size
def area2kwp(area):
    '''returns kwp provided by panel size'''
    return (
        area
        *settings['panel_info']['rated_capacity_kw']
        /settings['panel_info']['panel_area_sqm']
    )

# func to compute total panel area
def total_area(need,output):
    if output == 0:
        return 1000_000
    else:
        return need/output*1000

In [166]:
# find the optimal tilts, alignments and locations for the highest possible output or high roi

# imports
from d03_processing.cost_and_earnings import compute_finances, optimal_combination, setup_cost
from math import ceil
from d07_visualisation.create_geometries import create_polygons

@app.callback(
    Output('maxout_status','children'),
    Output('maxroi_status','children'),
    Input('output_status','children'), # trigger when XYZ
    State('process_id_store','data')
)
def get_need(status,process_id):
    if status == '':
        return '',''
    else:
        # start the timer
        starttime = perf_counter()
        # get the process id
        process_id = process_id[0]

        # load weather_df 
        weather_df = pd.read_parquet('../data/03_processed/weather_df_output_{}'.format(process_id))


        # aggregation dict, will be used to find the optimal tilts, alignments and locations
        # choose aggregation method for relevant columns
        sums = ['output']
        means = ['loss_snow','loss_temp','loss_soiling','loss_highvar']
        # construct a dict from the list
        agg_dict = {
            **dict.fromkeys(sums,'sum'),
            **dict.fromkeys(means,'mean')
        }
        # aggregate weather df with the aggregation dict, save with new name to keep the former weather_df as it is
        weather_df_max =(
            weather_df.copy()
            # groupby id_tuple tilt and alignment to compare between those
            .groupby(['id_tuple','tilt','alignment'])
            .agg(agg_dict)
        )
        # keep separate loss_df, will be added back to the df later on 
        loss_df = (
            weather_df_max.copy()
            # keep only the relevant columns
            .iloc[:,0:8]
            .drop('output',axis=1)
        )

        # save the tilt-alignment combination corresponding to the highest total output for each location: best
        max_output_index = list(
            weather_df_max.copy()
            .groupby(['id_tuple'])
            .agg({'output':'idxmax'})
            .output
        )

        # load the need_df
        need_df = pd.read_parquet('../data/02_intermediate/need_df_{}.pqt'.format(process_id))

        # create a dataframe with the highest possible output for each location
        max_out_df=(
            optimal_combination(
                weather_df,
                max_output_index,
                'output',
                need_df,loss_df
            )
        )

        # look up financial settings and the scale
        fin_settings = settings['financial']
        project_info = settings['project_info']
        
        # add col with the total area
        max_out_df.loc[:,'panel_area'] = project_info['total_panel_size'][scale]
        # compute financial info for the locations and tilt and alignment combinations
        max_out_df = compute_finances(
            max_out_df,
            settings['project_info']['max_kwp'][scale],
            fin_settings['kwh_cost'],
            compensation,
            output_col='output'
        )
        # round the output col
        max_out_df.output_total_daily= [
            round(output,2) for output in max_out_df.output_total_daily
        ]
        # add a col for the setup cost
        max_out_df.loc[:,'installation_cost'] = setup_cost(29.5)

        # add ratio between savings and installation cost
        max_out_df.loc[:,'savings_cost_ratio'] = [
            savings/cost
            for savings, cost
            in zip(
                max_out_df.savings_daily,
                max_out_df.installation_cost) 
        ]

        # calculate the installment payment if the power system would be partially (80%) financed by taking out a loan
        # loan is paid back over 20 years with 4.5% interest
        max_out_df.loc[:,'installment_payment_monthly'] = round(max_out_df.installation_cost*1000/197.6,2)
        # compute the real savings after the monthly installment
        max_out_df.loc[:,'savings_after_installment_monthly'] = [
            daily*30 - installment
            for daily, installment 
            in zip(
                max_out_df.savings_daily,
                max_out_df.installment_payment_monthly
            )
        ]
        # compute the yearly return on investment in comparison to the investment of 20% of the whole setup cost
        max_out_df.loc[:,'return_yearly'] = [
            round(12*savings/(investment*10/5),1)
            for savings, investment
            in zip(
                max_out_df.savings_after_installment_monthly,
                max_out_df.installation_cost
            )
        ]

        # use max out to create a geojson file with polygons to plot
        # copy the first col of max_out_df, content doesn't matter
        to_geojson = max_out_df[[max_out_df.columns[0]]].copy()
        # make sure the index consists of tuples
        to_geojson.index = [
            literal_eval(tup) 
            for tup in to_geojson.index
        ]
        to_geojson.index.name = 'id_tuple'
        # add a geometry col which contains polygons, these will be exported
        to_geojson = create_polygons(to_geojson).reset_index()
        print(to_geojson.columns)
        # set coordinate reference system
        crs = 'epsg:4326'
        # turn the dataframe into a geodataframe, this makes it possible to create a geojson file
        to_geojson = gpd.GeoDataFrame(to_geojson, crs=crs, geometry=to_geojson.geometry)
        # create new id_col for easier plotting
        to_geojson.loc[:,'id'] = to_geojson.id_tuple.astype(str)
        # create a geojson file
        polygon_json = gpd.GeoSeries(to_geojson.geometry).__geo_interface__
        # save the geojson
        path = '../references/polygons_'+process_id+'.json'
        with open(path,'w') as f:
            json.dump(polygon_json,f)


        # save max_out_df
        max_out_df.to_parquet('../data/03_processed/max_out_{}.pqt'.format(process_id))

        # stop the timer
        elapsed1 = perf_counter() -starttime
        # start a new timer
        starttime = perf_counter()

        # compute the optimal combination of tilt and alignment to reach the requested ratio of 
        # self sufficient half-days with the minimal total panel area
        # aggregrate based on the agg dict, join energy need data to weather_df
        weather_df = (
            weather_df
            # groupby by tilt and alignment to find the best combination, groupby date and pm to compare half days
            .groupby(['id_tuple','date','pm','alignment','tilt'])
            # same aggregation as for the max out df
            .agg(agg_dict)
            .reset_index()
            .join(need_df,on=['date','pm'])
        )
        # compute the needed total area which would lead to the requested ratio of self sufficient half-days
        weather_df.loc[:,'area_indicator'] = [total_area(need,output) for need,output in zip(weather_df.need,weather_df.output)]

        # create an indicator_df which stores what area would be required to reach the requested ratio of self sufficient half-days
        indicator_df = (
            weather_df.copy()
            .set_index('id_tuple')
            # sort so the custom aggregator works fine
            .sort_values(['tilt','alignment','area_indicator'])
            .groupby(['id_tuple','tilt','alignment'])
            # find the requested percentile for each tilt-alignment combination
            .agg({'area_indicator':agg_percentile_finder})
        )

        # save the index of the rows with the smallest area for each location
        # this index includes the tilt and alignment values
        smallest_size_idx = list(
            indicator_df
            # find the index with the lowest area indicator per location
            # index is made up of tilt and alignment
            .groupby(['id_tuple'])
            .agg('idxmin')
            .area_indicator
        )

        # get the df optimized for self sufficiency 
        smallest_size_df=(
            optimal_combination(
                weather_df,
                smallest_size_idx,
                'output',
                need_df,loss_df,indicator_df
            )
        )

        # compute financial data for the smallest panel installations
        smallest_size_df = compute_finances(
            smallest_size_df,
            settings['project_info']['max_kwp'][scale],
            fin_settings['kwh_cost'],
            compensation,
            output_col='output',
            panel_size_col='area_indicator'
        )

        # join indicator df to smallest_size_df to get the info about optimal tilt and alignment combinations
        smallest_size_df=smallest_size_df.join(indicator_df.loc[smallest_size_idx])
        # add installation cost
        smallest_size_df.loc[:,'installation_cost'] = [
            setup_cost(area2kwp(size))
            for size
            in smallest_size_df.area_indicator
        ]

        # add ratio between savings and installation cost
        smallest_size_df.loc[:,'savings_cost_ratio'] = [
            savings/cost
            for savings, cost
            in zip(
                smallest_size_df.savings_daily,
                smallest_size_df.installation_cost) 
        ]

        # calculate the installment payment if the power system would be partially (80%) financed by taking out a loan
        # loan is paid back over 20 years with 4.5% interest
        smallest_size_df.loc[:,'installment_payment_monthly'] = round(smallest_size_df.installation_cost*1000/197.6,2)
        # compute the real savings after the monthly installment
        smallest_size_df.loc[:,'savings_after_installment_monthly'] = [
            daily*30 - installment
            for daily, installment 
            in zip(
                smallest_size_df.savings_daily,
                smallest_size_df.installment_payment_monthly
            )
        ]
        # compute the yearly return on investment in comparison to the investment of 20% of the whole setup cost
        smallest_size_df.loc[:,'return_yearly'] = [
            round(12*savings/(investment*10/5),1)
            for savings, investment
            in zip(
                smallest_size_df.savings_after_installment_monthly,
                smallest_size_df.installation_cost
            )
        ]

        # reset and re-set the index so that only id_tuple is in the index
        smallest_size_df = smallest_size_df.reset_index().set_index('id_tuple')
        # save smallest_size_df 
        smallest_size_df.to_parquet('../data/03_processed/max_roi_{}.pqt'.format(process_id))
        elapsed2 = perf_counter()-starttime

        return (
            ('94% done - Highest possible output for each location analysed in '+f'{elapsed1:0.2f} seconds'),
            ('95% done - Highest possible ROI for each location analysed in '+f'{elapsed1:0.2f} seconds'),
        )

In [167]:
# show the plots

# imports
import plotly.express as px

@app.callback(
    Output('plotted_status','children'), # inform the user when the plots are available
    # output the plots
    Output('map_graph','figure'),
   
    Input('maxout_status','children'), # trigger when the data is analyzed
    Input('to_plot_radio','value'), # also trigger when a new output is chosen
    State('process_id_store','data')
)
def get_need(status,to_plot,process_id):
    if status == '':
        return '', no_fig
    else:
        # start the timer
        starttime = perf_counter()
        # get the process id
        process_id = process_id[0]

        # load the dfs and the geodata
        maxout_df = pd.read_parquet('../data/03_processed/max_out_{}.pqt'.format(process_id)).reset_index().reset_index()
        maxroi_df = pd.read_parquet('../data/03_processed/max_roi_{}.pqt'.format(process_id)).reset_index().reset_index()
        with open('../references/polygons_{}.json'.format(process_id), 'r') as f:
            geojson = json.load(f)
        # create dict to choose correct df to plot
        # dict includes a list for each to_plot value, first element is the df, second is the name of the col which should be plotted
        to_plot_dict = {
            'maxout':[maxout_df,'output_total_daily'],
            'maxroi':[maxroi_df,'return_yearly'],
        }

        # begin by creating the map
        map_fig = px.choropleth_mapbox(to_plot_dict[to_plot][0], geojson=geojson,
                                locations='index', color=to_plot_dict[to_plot][1],
                                color_continuous_scale="bluered",
                                #    range_color=(0.1, .11),
                                
                                mapbox_style="open-street-map",
                                zoom=9, center = {
                                    "lat": (
                                        # get the middle latitude
                                        geojson['features'][int(len(geojson['features'])/2)]['bbox'][1]
                                    ),
                                    "lon":(
                                        geojson['features'][int(len(geojson['features'])/2)]['bbox'][0]
                                    )
                                },
                                labels={
                                    'output_total_daily':'Maximum electricity<br>output per day<br>in kWh',
                                    'return_yearly':'Yearly return<br>on investment<br>in percent'
                                },
                                opacity=0.65,
                                # hover_data={
                                #     'output_total_daily':'Maximum electricity output per day',
                                #     'return_yearly':'Yearly return on investment'
                                # },
                                height=1080, width=1920
                                )
        map_fig.update_layout(
            margin={"r":0,"t":0,"l":0,"b":0}
        )

        hist_fig = px.scatter(
            to_plot_dict[to_plot][0],
            x='loss_temp',
            y='loss_soiling',
            color='loss_highvar',
            color_continuous_scale="bluered",
            height = 600
        )

        # stop the timer
        elapsed = perf_counter() -starttime
        return ('100% done - Plots generated in '+f'{elapsed:0.2f} seconds'), map_fig

In [168]:
# run the server
# server can also be accessed via browser (http://127.0.0.1:8050/)
app.run_server(mode='jupyterlab',debug=True,port=8050)

C:\Users\Techie\anaconda3\envs\mapenv\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



clicks reset
locref triggered but no update
get weather triggered, return empty string
loc ref block triggered, nothing returned
no weather data, waiting
locref triggered and work done
available weather data will be loaded. change use_available or process_id to request new data
weather checked, id: ['duesseldorf_200']
ids df is a <class 'pandas.core.frame.DataFrame'>
pre point
tuple is in str
now its a tuple
geo created
geoblock done
blocks updated
blocks by locref done
weather pivoted
timer started for requests
topo available
topo block started
Index(['id_tuple', 'tilt', 'alignment', 'date', 'pm', 'output', 'need',
       'loss_snow', 'loss_temp', 'loss_soiling', 'loss_highvar', 'panel_area'],
      dtype='object')
Index(['id_tuple', 'loss_snow', 'geometry'], dtype='object')
Index(['id_tuple', 'tilt', 'alignment', 'date', 'pm', 'output', 'need',
       'loss_snow', 'loss_temp', 'loss_soiling', 'loss_highvar',
       'area_indicator'],
      dtype='object')
